# Gönül AYCI - 2016800003

## Artificial Neural Networks - Assigment-II: Logistic Regression

Inputs are 16x16 binary images and there are 10 classes. <br>
You will implement linear logistic regression. <br>
Show how training and test misclassifications change as training proceeds and report the final confusion matrices on training and test data.

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import itertools

### Read data from given test and train files

In [2]:
test_file = "testcom.txt"
train_file = "traicom.txt"

test_lines = [line.rstrip('\n') for line in open(test_file)]
train_lines = [line.rstrip('\n') for line in open(train_file)]

#### Preparation of train data

I added 1's into my x_train data in here rather than at the creation of weight zero. So, <br> 
x_train.shape = (600, 257) <br>
r_train.shape = (600, 10) <br>

In [3]:
x_train = []
r_train = []

for i in range(0, len(train_lines), 17):
    train_data = train_lines[i:i+17]
    join_data = '1' + (''.join(train_data))

    x_train.append([int(a) for a in join_data[0:-1]])
    r = np.zeros(10)
    r[int(join_data[-1])] += 1
    r_train.append(r)
    
x_train = np.asarray(x_train)
r_train = np.asarray(r_train)

#### Preparation of test data

I added 1's into my x_test data in here rather than at the creation of weight zero. So, <br> 
x_test.shape = (600, 257) <br>
r_test.shape = (600, 10) <br>

In [4]:
x_test = []
r_test = []

for i in range(0, len(test_lines), 17):
    test_data = test_lines[i:i+17]
    join_data ='1' + (''.join(test_data))
    
    x_test.append([int(a) for a in join_data[0:-1]])
    r = np.zeros(10)
    r[int(join_data[-1])] += 1
    r_test.append(r)
    
x_test = np.asarray(x_test)
r_test = np.asarray(r_test)

### Assign values 

In [5]:
eta = .001 # learning rate
nEpochs = 50 # # of epochs
nClass = 10 # # of class
N = 257 # # of features : 256 +1
instance = 600 # # of samples

### Creation 10 by 257 weight matrix

In [6]:
#w0 = np.ones((nClass,1))

initialValue = 0.001
w = np.zeros((nClass, N)) # (10,257)

for r in range(nClass):
    for c in range(N):
        w[r][c] = random.uniform(-initialValue, +initialValue)

### Definition of Softmax function

In [7]:
def softmax(input_o):
    input_o -= np.max(input_o, axis=0)
    return (np.exp(input_o) / np.sum(np.exp(input_o), axis=0))

### Definition of Confusion Matrix

In [8]:
def confusion_matrix_func(w, x, r):
    match = 0
    mismatch = 0
    
    confusion_matrix = [[0 for x in range(nClass)] for y in range(nClass)] 
    confusion_matrix = np.asarray(confusion_matrix)
    
    results = softmax(np.dot(w, np.transpose(x)))

    for i, res in enumerate(np.transpose(results)):
        prediction = res.argmax()
        true = r[i,:].argmax()
        
        if prediction == true:
            confusion_matrix[prediction, prediction] += 1
            match += 1
        else:
            confusion_matrix[true, prediction] += 1
            mismatch += 1
            
    return match, mismatch, confusion_matrix

### Logistic Regression part

In [9]:
print("Training and Test Misclassification Changes:", '\n')

y_train = np.zeros(nClass)

for epoch in range(nEpochs):

    random.shuffle([x for x in range(instance)])
    
    for i in range(instance):
        x_tr = x_train[i].reshape(N, 1)
        r_tr = r_train[i].reshape(nClass, 1)
        
        input_sum = np.zeros((nClass, 1))
        input_sum = np.dot(w,x_tr)
        input_sum = input_sum.reshape(nClass, 1)
        y_train = softmax(input_sum)
        
        deltaW = eta * np.dot(r_tr - y_train, np.transpose(x_tr))
        w += deltaW
                    
    eta *= .999 
        
    match, mismatch, confusion_matrix = confusion_matrix_func(w, x_train, r_train)
    match2, mismatch2, confusion_matrix = confusion_matrix_func(w, x_test, r_test)
    
    print("EPOCH: ", str(epoch+1))
    print("Number of Mismatch for train data: ", str(mismatch))
    print("Number of Match for train data: ", str(match))
    print("Number of Mismatch for test data : ", str(mismatch2))
    print("Number of Match for test data : ", str(match2))
    print()


Training and Test Misclassification Changes: 

EPOCH:  1
Number of Mismatch for train data:  95
Number of Match for train data:  505
Number of Mismatch for test data :  126
Number of Match for test data :  474

EPOCH:  2
Number of Mismatch for train data:  67
Number of Match for train data:  533
Number of Mismatch for test data :  88
Number of Match for test data :  512

EPOCH:  3
Number of Mismatch for train data:  61
Number of Match for train data:  539
Number of Mismatch for test data :  75
Number of Match for test data :  525

EPOCH:  4
Number of Mismatch for train data:  52
Number of Match for train data:  548
Number of Mismatch for test data :  70
Number of Match for test data :  530

EPOCH:  5
Number of Mismatch for train data:  44
Number of Match for train data:  556
Number of Mismatch for test data :  67
Number of Match for test data :  533

EPOCH:  6
Number of Mismatch for train data:  44
Number of Match for train data:  556
Number of Mismatch for test data :  60
Number of Ma

### Get created confusion matrices

In [10]:
_, _, confusion_matrix_train = confusion_matrix_func(w, x_train, r_train)
_, _, confusion_matrix_test = confusion_matrix_func(w, x_test, r_test)

### Plot confusion matrices for training and test data